In [ ]:
# !pip install numpy pandas statsmodels matplotlib seaborn scikit-learn
!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

In [4]:
# === 1. Load data ===
file_path = "startDates.csv"

if file_path.endswith(".csv"):
    df_raw = pd.read_csv(file_path)
else:
    df_raw = pd.read_excel(file_path)

df_raw.head()

,start_date
0,2025-03-29
1,2025-05-23
2,2025-07-01
3,2025-07-30
4,2025-02-20


In [5]:
# Preprocess dates
df = df_raw.copy()
df['start_date'] = pd.to_datetime(df['start_date'])
df = df.sort_values('start_date').reset_index(drop=True)
df.head()

,start_date
0,2021-09-29
1,2021-11-09
2,2021-12-15
3,2022-01-11
4,2022-02-09


In [6]:
# === 2. Compute cycle lengths (in days) ===
df['cycle_length'] = df['start_date'].diff().dt.days
cycle_lengths = df['cycle_length'].dropna().astype(int)

df.head()
# cycle_lengths.head()

,start_date,cycle_length
0,2021-09-29,NaN
1,2021-11-09,41.0
2,2021-12-15,36.0
3,2022-01-11,27.0
4,2022-02-09,29.0


EDA for trend and seasonality, stationary series test

In [ ]:
# 1. Plot the raw data
df["cycle_length"].plot(title="Cycle Length Over Time", figsize=(10,5))
plt.show()

# 2. Rolling mean & std (to check trend/volatility)
df["cycle_length"].rolling(window=5).mean().plot(label="Rolling Mean")
df["cycle_length"].rolling(window=5).std().plot(label="Rolling Std")
plt.legend()
plt.show()

# 3. Autocorrelation plots (to check seasonality patterns)
plot_acf(df["cycle_length"], lags=20)
plt.show()

plot_pacf(df["cycle_length"], lags=20)
plt.show()

# 4. Stationarity test (ADF test)
result = adfuller(df["cycle_length"].dropna())
print("ADF Statistic:", result[0])
print("p-value:", result[1])


final comparision over models

In [44]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

# === 1. Load Data ===
file_path = "startDates.csv"  # or "cycles.xlsx"
if file_path.endswith(".csv"):
    df = pd.read_csv(file_path)
else:
    df = pd.read_excel(file_path)

df['start_date'] = pd.to_datetime(df['start_date'])
df = df.sort_values('start_date').reset_index(drop=True)
df['cycle_length'] = df['start_date'].diff().dt.days
cycle_lengths = df['cycle_length'].dropna().reset_index(drop=True)

# === 2. Train-Test Split ===
train_size = int(len(cycle_lengths) * 0.8)
train, test = cycle_lengths[:train_size], cycle_lengths[train_size:]

# Function to compute metrics
def compute_metrics(y_true, y_pred, tolerance=3):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    accuracy = 100 - mape
    within_tol = (np.abs(y_true - y_pred) <= tolerance).mean() * 100
    return mae, rmse, mape, accuracy, within_tol

# === 3. Exponential Smoothing ===
es_model = ExponentialSmoothing(train, trend=None, seasonal=None)
es_fit = es_model.fit()
es_forecast_test = es_fit.forecast(len(test))
mae_es, rmse_es, mape_es, acc_es, tol3_es = compute_metrics(test, es_forecast_test)

es_next_length = round(es_fit.forecast(1).iloc[0])
es_last_start = df['start_date'].iloc[-1]
es_window = 7  # ±7 days
es_next_start = es_last_start + pd.Timedelta(days=es_next_length)
es_next_range = (es_next_start - pd.Timedelta(days=es_window), es_next_start + pd.Timedelta(days=es_window))

# === 4. ARIMA ===
arima_model = ARIMA(train, order=(1,0,1))
arima_fit = arima_model.fit()
arima_forecast_test = arima_fit.forecast(len(test))
mae_ar, rmse_ar, mape_ar, acc_ar, tol3_ar = compute_metrics(test, arima_forecast_test)

arima_next_length = round(arima_fit.forecast(1).iloc[0])
arima_window = 7  # ±7 days
arima_next_start = es_last_start + pd.Timedelta(days=arima_next_length)
arima_next_range = (arima_next_start - pd.Timedelta(days=arima_window), arima_next_start + pd.Timedelta(days=arima_window))

# === 5. Simple Moving Average ===
window = 3
ma_forecast_test = []
train_list = train.tolist()
for i in range(len(test)):
    ma_pred = np.mean(train_list[-window:])
    ma_forecast_test.append(ma_pred)
    train_list.append(test.iloc[i])
ma_forecast_test = np.array(ma_forecast_test)
mae_ma, rmse_ma, mape_ma, acc_ma, tol3_ma = compute_metrics(test, ma_forecast_test)

ma_next_length = round(np.mean(cycle_lengths[-window:]))
ma_window = 5  # ±5 days
ma_next_start = es_last_start + pd.Timedelta(days=ma_next_length)
ma_next_range = (ma_next_start - pd.Timedelta(days=ma_window), ma_next_start + pd.Timedelta(days=ma_window))

# === 6. Weighted Moving Average ===
weights = np.array([0.6, 0.3, 0.1])
wma_forecast_test = []
train_list = train.tolist()
for i in range(len(test)):
    wma_pred = np.dot(train_list[-3:], weights)
    wma_forecast_test.append(wma_pred)
    train_list.append(test.iloc[i])
wma_forecast_test = np.array(wma_forecast_test)
mae_wma, rmse_wma, mape_wma, acc_wma, tol3_wma = compute_metrics(test, wma_forecast_test)

wma_next_length = round(np.dot(cycle_lengths[-3:], weights))
wma_window = 5  # ±5 days
wma_next_start = es_last_start + pd.Timedelta(days=wma_next_length)
wma_next_range = (wma_next_start - pd.Timedelta(days=wma_window), wma_next_start + pd.Timedelta(days=wma_window))

# === 7. Print Metrics & Next Cycle Predictions with Windows ===
print("Model Performance on Test Data")
print("""ES: MAE={:.2f}, RMSE={:.2f}, MAPE={:.2f}%, Accuracy={:.2f}%, ±3-day={:.2f}%
ARIMA: MAE={:.2f}, RMSE={:.2f}, MAPE={:.2f}%, Accuracy={:.2f}%, ±3-day={:.2f}%
MA: MAE={:.2f}, RMSE={:.2f}, MAPE={:.2f}%, Accuracy={:.2f}%, ±3-day={:.2f}%
WMA: MAE={:.2f}, RMSE={:.2f}, MAPE={:.2f}%, Accuracy={:.2f}%, ±3-day={:.2f}%"""
      .format(mae_es, rmse_es, mape_es, acc_es, tol3_es,
              mae_ar, rmse_ar, mape_ar, acc_ar, tol3_ar,
              mae_ma, rmse_ma, mape_ma, acc_ma, tol3_ma,
              mae_wma, rmse_wma, mape_wma, acc_wma, tol3_wma))

print("\nNext Cycle Predictions with Forecast Windows")
print(f"Exponential Smoothing: {es_next_start.date()} ± {es_window} days ({es_next_range[0].date()} to {es_next_range[1].date()})")
print(f"ARIMA: {arima_next_start.date()} ± {arima_window} days ({arima_next_range[0].date()} to {arima_next_range[1].date()})")
print(f"Moving Average: {ma_next_start.date()} ± {ma_window} days ({ma_next_range[0].date()} to {ma_next_range[1].date()})")
print(f"Weighted Moving Average: {wma_next_start.date()} ± {wma_window} days ({wma_next_range[0].date()} to {wma_next_range[1].date()})")

Model Performance on Test Data
ES: MAE=10.26, RMSE=13.13, MAPE=21.51%, Accuracy=78.49%, ±3-day=12.50%
ARIMA: MAE=9.10, RMSE=12.00, MAPE=18.97%, Accuracy=81.03%, ±3-day=12.50%
MA: MAE=11.50, RMSE=13.13, MAPE=27.55%, Accuracy=72.45%, ±3-day=25.00%
WMA: MAE=12.50, RMSE=13.73, MAPE=30.05%, Accuracy=69.95%, ±3-day=12.50%

Next Cycle Predictions with Forecast Windows
Exponential Smoothing: 2025-09-01 ± 7 days (2025-08-25 to 2025-09-08)
ARIMA: 2025-09-05 ± 7 days (2025-08-29 to 2025-09-12)
Moving Average: 2025-09-09 ± 5 days (2025-09-04 to 2025-09-14)
Weighted Moving Average: 2025-09-16 ± 5 days (2025-09-11 to 2025-09-21)


Add more features

In [ ]:
from sklearn.linear_model import LinearRegression

# === 2. Feature Engineering ===
avg_cycle_length = cycle_lengths.mean()  # average cycle length
avg_period_duration = 5  # given period lasts 5 days

# Create features for regression
# Use previous cycle length as feature + engineered features
X = pd.DataFrame({
    'prev_cycle_length': cycle_lengths.shift(1),
    'avg_cycle_length': avg_cycle_length,
    'avg_period_duration': avg_period_duration
})
Y = cycle_lengths

# Drop first row with NaN
X = X.dropna().reset_index(drop=True)
Y = Y[1:].reset_index(drop=True)

# === 3. Train-Test Split ===
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

# === 4. Linear Regression Model ===
model = LinearRegression()
model.fit(X_train, Y_train)

# Predict on test set
y_pred = model.predict(X_test)

# === 5. Metrics ===
def compute_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    accuracy = 100 - mape
    return mae, rmse, mape, accuracy

mae, rmse, mape, acc = compute_metrics(Y_test, y_pred)
print(f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}%, Accuracy={acc:.2f}%")

# === 6. Predict Next Cycle ===
last_cycle = cycle_lengths.iloc[-1]
X_next = pd.DataFrame({
    'prev_cycle_length': [last_cycle],
    'avg_cycle_length': [avg_cycle_length],
    'avg_period_duration': [avg_period_duration]
})
next_cycle_length = model.predict(X_next)[0]
next_cycle_date = df['start_date'].iloc[-1] + pd.Timedelta(days=round(next_cycle_length))
print(f"Predicted next cycle start date: {next_cycle_date.date()} ({round(next_cycle_length)} days)")

MAE=11.87, RMSE=14.65, MAPE=25.18%, Accuracy=74.82%


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- avg_cycle_length
- avg_period_duration


In [11]:
from sklearn.ensemble import RandomForestRegressor

# === 2. Feature Engineering ===
avg_cycle_length = cycle_lengths.mean()  # average cycle length
avg_period_duration = 5  # given period lasts 5 days

# Create features for regression
X = pd.DataFrame({
    'prev_cycle_length': cycle_lengths.shift(1),
    # 'avg_cycle_length': avg_cycle_length,
    # 'avg_period_duration': avg_period_duration
})
Y = cycle_lengths

# Drop first row with NaN
X = X.dropna().reset_index(drop=True)
Y = Y[1:].reset_index(drop=True)

# === 3. Train-Test Split ===
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

# === 4. Random Forest Regressor ===
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)

# Predict on test set
y_pred = model.predict(X_test)

# === 5. Metrics ===
def compute_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    accuracy = 100 - mape
    return mae, rmse, mape, accuracy

mae, rmse, mape, acc = compute_metrics(Y_test, y_pred)
print(f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}%, Accuracy={acc:.2f}%")

# === 6. Predict Next Cycle ===
last_cycle = cycle_lengths.iloc[-1]
X_next = pd.DataFrame({
    'prev_cycle_length': [last_cycle],
    # 'avg_cycle_length': [avg_cycle_length],
    # 'avg_period_duration': [avg_period_duration]
})
next_cycle_length = model.predict(X_next)[0]
next_cycle_date = df['start_date'].iloc[-1] + pd.Timedelta(days=round(next_cycle_length))
print(f"Predicted next cycle start date: {next_cycle_date.date()} ({round(next_cycle_length)} days)")


MAE=13.28, RMSE=17.46, MAPE=28.45%, Accuracy=71.55%
Predicted next cycle start date: 2025-08-28 (29 days)


In [ ]:


# === 2. Feature Engineering ===
avg_cycle_length = cycle_lengths.mean() # average cycle length
avg_period_duration = 5 # given period lasts 5 days


# Create lag features: previous 3 cycles
lag_features = 3
X = pd.DataFrame({
'lag1': cycle_lengths.shift(1),
'lag2': cycle_lengths.shift(2),
'lag3': cycle_lengths.shift(3),
'avg_cycle_length': avg_cycle_length,
'avg_period_duration': avg_period_duration
})
Y = cycle_lengths


# Drop rows with NaN due to lagging
X = X.dropna().reset_index(drop=True)
Y = Y[lag_features:].reset_index(drop=True)


# === 3. Train-Test Split ===
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]


# === 4. XGBoost Regressor ===
model = XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, Y_train)


# Predict on test set
y_pred = model.predict(X_test)


# === 5. Metrics ===
def compute_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    accuracy = 100 - mape
    return mae, rmse, mape, accuracy


mae, rmse, mape, acc = compute_metrics(Y_test, y_pred)
print(f"MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}%, Accuracy={acc:.2f}%")


# === 6. Predict Next Cycle ===
last_cycles = cycle_lengths.iloc[-3:].tolist() # last 3 cycles
X_next = pd.DataFrame({
'lag1': [last_cycles[-1]],
'lag2': [last_cycles[-2]],
'lag3': [last_cycles[-3]],
'avg_cycle_length': [avg_cycle_length],
'avg_period_duration': [avg_period_duration]
})
next_cycle_length = model.predict(X_next)[0]
next_cycle_date = df['start_date'].iloc[-1] + pd.Timedelta(days=round(next_cycle_length))
print(f"Predicted next cycle start date: {next_cycle_date.date()} ({round(next_cycle_length)} days)")

NameError: name 'XGBRegressor' is not defined